# Initializing SSH Connection, Motors, and pH Measurements

In [ ]:
#pip install opencv-python
#pip uninstall serial
#pip install pyserial
#!pip install rpyc
#!pip install python-ternary

import paramiko # for interacting over ssh
import rpyc # lets you interact with remote devices as if local
print( rpyc.__version__)
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import GPy
import math
from numpy.random import seed
import cv2 # computer vision package
import sys, os
seed(12345)
# name is ev3dev on for (PuTTy)
# SSH connection over wifi
host = "192.168.0.102"
host2 = "192.168.0.101"
port = 22
username = "pi"
password = "raspberry"
cmd_to_execute = ". ~/rpyc_server.sh" # starts the rpyc server on the lego system

In [ ]:
# initialize ssh connection and start the rpyc server
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, username=username, password=password)

In [ ]:
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(cmd_to_execute)
print(ssh_stdin,ssh_stdout,ssh_stderr)

In [ ]:
# initialize ssh connection and start the rpyc server
ssh2 = paramiko.SSHClient()
ssh2.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh2.connect(host2, username=username, password=password)

In [ ]:
ssh_stdin, ssh_stdout, ssh_stderr = ssh2.exec_command(cmd_to_execute)
print(ssh_stdin,ssh_stdout,ssh_stderr)

In [ ]:
ssh_stderr.read()

In [ ]:
# Connect to and Define Buildhats 
conn = rpyc.classic.connect(host=host)
r_buildhat = conn.modules.buildhat # XY Control

In [ ]:
conn2 = rpyc.classic.connect(host=host2)
r_buildhat2 = conn2.modules.buildhat  # Cart Controls

In [ ]:
#Motors and Sensors (BH1)
motorX = r_buildhat.Motor("D")        # X axis
sensorX = r_buildhat.ForceSensor("C") # X axis sensor
motorY = r_buildhat.Motor("B")        # Y axis
sensorY = r_buildhat.ForceSensor("A") # Y axis sensor

#Motors and Sensors (BH2)
motorpH = r_buildhat2.Motor("B")      # pH control 
motorS = r_buildhat2.Motor("D")       # syringe control (Z axis movement)
motorV =  r_buildhat2.Motor("C")      # volume control (plunger)

In [ ]:
#---------------Import Serial to Monitor pH from Arduino port ---------------
import serial 
ser = serial.Serial('COM9')

In [ ]:
ser.close() # use this function to keep serial monitor closed until called

# Calibration - Motors and Sensors

In [ ]:
#motorV.run_for_degrees(-600)   #+ is up,- is down  - Volume Control

In [ ]:
#motorS.run_for_degrees(250)  #-250 down, 250 up

In [ ]:
#motorpH.run_for_degrees(-325)  #325 down, 325 up

In [ ]:
def XYReset(a,b):
    motorX.start(50)
    sensorX.wait_until_pressed(force = 0.02)
    motorX.stop()
    motorX.run_for_degrees(-a)
    motorY.start(-30)
    sensorY.wait_until_pressed(force = 0.05)
    motorY.stop()
    motorY.run_for_degrees(b)
    x_start = motorX.get_position()
    y_start = motorY.get_position()
    return x_start, y_start

In [ ]:
def system_check():           
    motorpH.run_for_degrees(325)  #325 down, -325 up
    motorpH.run_for_degrees(-325)
    motorS.run_for_degrees(-300)  #-300 down, 300 up
    motorS.run_for_degrees(300)  #-300 down, 300 up
    motorV.run_for_degrees(600)   #+ is up,- is down 
    motorV.run_for_degrees(-600)   #+ is up,- is down 
    

## Note: 
Make sure pH sensor is at top, syringe is at top, and plunger is fully down

In [ ]:
a = -20
b = 20
x_start, y_start = XYReset(a,b)
#system_check()

In [ ]:
s_full_down = -300
x_offset = -138

In [ ]:
# Syringe X Offsets for Wells
x_offset = -138  #distance between wells in x-direction
x_start, y_start = XYReset(a,b)

#Testing after XY Reset
for i in range (5):
    motorX.run_for_degrees(x_offset)
    time.sleep(1)
    
regionX_offset = -235  # slight offset as we enter a new sample region
motorX.run_for_degrees(regionX_offset)
time.sleep(1)

for i in range (2):
    motorX.run_for_degrees(x_offset)
    time.sleep(1)
    
# In Total we can reach 9 cells in this direction

In [ ]:
# Syringe Y Offsets for Wells
y_offset = 135  #distance between wells in x-direction
x_start, y_start = XYReset(a,b)

#Testing after XY Reset
for i in range (3):
    motorY.run_for_degrees(y_offset)
    time.sleep(1)

regionY_offset = 200  # slight offset as we enter a new sample region
motorY.run_for_degrees(regionY_offset)
time.sleep(1)

for i in range (3):
    motorY.run_for_degrees(y_offset)
    time.sleep(1)
    
# In Total we can reach 7 cells in this direction
# This allow for 63 total cells to reach

In [ ]:
# pH Sensor XY offset
pH_Y_offset = 210
pH_X_offset = 0
pH_full_down = 350

motorY.run_for_degrees(pH_Y_offset)
motorX.run_for_degrees(pH_X_offset)
time.sleep(2)

motorpH.run_for_degrees(pH_full_down)  #370 down, -370 up
time.sleep(2)
motorpH.run_for_degrees(-pH_full_down)



motorY.run_for_degrees(-pH_Y_offset)
motorX.run_for_degrees(-pH_X_offset)
time.sleep(2)


In [ ]:
#pH_full_down = 320

In [ ]:
def sgoto(row,col):
    originX_offset = motorX.get_position() - x_start
    if row <=5:
        positionX = row * x_offset
    elif row > 5:
        positionX = (row - 1) * x_offset + regionX_offset
    motorX.run_for_degrees(positionX - originX_offset)
    
    originY_offset = motorY.get_position() - y_start
    if col <= 3:
        positionY = col * y_offset
    elif col > 3: 
        positionY = (col - 1) * y_offset + regionY_offset
    motorY.run_for_degrees(positionY-originY_offset)
    

In [ ]:
sgoto(0,0)

In [ ]:
def pgoto(row,col):
    originX_offset = motorX.get_position() - x_start
    if row <=5:
        positionX = row * x_offset + pH_X_offset
    elif row > 5:
        positionX = (row - 1) * x_offset + regionX_offset + pH_X_offset
    motorX.run_for_degrees(positionX - originX_offset)
    
    originY_offset = motorY.get_position() - y_start
    if col <= 3:
        positionY = col * y_offset + pH_Y_offset
    elif col > 3: 
        positionY = (col - 1) * y_offset + regionY_offset + pH_Y_offset
    motorY.run_for_degrees(positionY-originY_offset)
    

In [ ]:
pgoto(5,3)

In [ ]:
def sAcid():
    originX_offset = motorX.get_position() - x_start
    positionX = 0
    motorX.run_for_degrees(positionX - originX_offset)
    
    originY_offset = motorY.get_position() - y_start
    positionY = 1250
    motorY.run_for_degrees(positionY - originY_offset)

In [ ]:
sAcid()

In [ ]:
def sBase():
    originX_offset = motorX.get_position() - x_start
    positionX = -400
    motorX.run_for_degrees(positionX - originX_offset)
    
    originY_offset = motorY.get_position() - y_start
    positionY = 1250
    motorY.run_for_degrees(positionY - originY_offset)

In [ ]:
sBase()

In [ ]:
def pClean():
    shake = 30                         
    sgoto(0,0)
    motorpH.run_for_degrees(pH_full_down)  #370 down, -370 up
    
    for i in range (4):                     #stir in water to clean
        motorpH.run_for_degrees(-shake)   
        motorpH.run_for_degrees(shake)
        
    motorpH.run_for_degrees(-pH_full_down)
    
    for i in range (4):                     # get droplets off
        motorpH.run_for_degrees(shake)
        motorpH.run_for_degrees(-shake)
    

In [ ]:
pClean()

In [ ]:
# pH Measuring Functions
def report_pH(t):                                #read the pH from the serial monitor
    ser.open()
    for i in range(t):
        x = ser.readline()
        x = x.decode()
        x = float(x.strip())
        time.sleep(1)
    ser.close()
    return x

def pH_measure(t):                               #encapsulating pH measurement function 
    motorpH.run_for_degrees(pH_full_down)
    shake = 30             
    for i in range (7):                         # for stirring
        motorpH.run_for_degrees(-shake)
        motorpH.run_for_degrees(shake)
    x = report_pH(t)
    motorpH.run_for_degrees(-pH_full_down)
    
    for i in range (3):                         # for shaking off droplets
        motorpH.run_for_degrees(shake)
        motorpH.run_for_degrees(-shake)
    
    print('pH is Measured as:', x)
    return x

def f(x):                                     # for simulating data
    f = 4.756 - np.log10(x) 
    return f

In [ ]:
pgoto(0,0)
pH_measure(20)

In [ ]:
x_start, y_start = XYReset(a,b)
print(x_start, y_start)

In [ ]:
#Test edge cases (only using first 20 cells for this experiment)
sgoto(5,3)
pgoto(5,3)

motorpH.run_for_degrees(pH_full_down)  #370 down, -370 up
motorpH.run_for_degrees(-pH_full_down)

In [ ]:

pgoto(0,0)
motorpH.run_for_degrees(pH_full_down)  #370 down, -370 up
motorpH.run_for_degrees(-pH_full_down)

In [ ]:
motorS.run_for_degrees(-s_full_down)

In [ ]:
motorV.run_for_degrees(-deg3)

# Experiment Setup

In [ ]:
# Best to Calibrate/Check these degrees correspond with volumes...

deg1 = 105
deg2 = 190
deg3 = 260
deg4 = 340
deg5 = 425
deg6 = 505
deg7 = 595
deg05 = 60  



def acid_dep(deg,row,col):
    sAcid()
    motorS.run_for_degrees(s_full_down)
    motorV.run_for_degrees(deg)
    motorS.run_for_degrees(-s_full_down)
    sgoto(row,col)
    motorV.run_for_degrees(-deg)

def base_dep(deg,row,col):
    sBase()
    motorS.run_for_degrees(s_full_down)
    motorV.run_for_degrees(deg)
    motorS.run_for_degrees(-s_full_down)
    sgoto(row,col)
    motorV.run_for_degrees(-deg)
    
def make_measure_clean(next_idx, next_pct_acid, next_pct_base, count):
    #--------------------alter this code based on your experimental well design setup -------------------------------#
    if count <= 5:         
        row = count
        col = 0
    elif count <= 11:        
        row = count - 6
        col = 1
    elif count <= 17:       
        row = count - 12
        col = 2 
    elif count <= 23:        
        row = count - 18
        col = 3
    #--------------------Creating Sample of Desired Composition ------------------------------------------------#
    acid_vol = next_pct_acid * 2.0        #volume of acid needed in mL
    acid_reps = np.rint(acid_vol * 20)
    while acid_reps > 0.1:
        if acid_reps >= 14: 
            acid_dep(deg7,row,col)
            acid_reps -= 14
        elif acid_reps >= 12:
            acid_dep(deg6,row,col)
            acid_reps -= 12
        elif acid_reps >= 10:
            acid_dep(deg5,row,col)
            acid_reps -= 10
        elif acid_reps >= 8:
            acid_dep(deg4,row,col)
            acid_reps -= 8
        elif acid_reps >= 6:
            acid_dep(deg3,row,col)
            acid_reps -= 6
        elif acid_reps >= 4:
            acid_dep(deg2,row,col)
            acid_reps -= 4
        elif acid_reps >= 2:
            acid_dep(deg1,row,col)
            acid_reps -= 2
        elif acid_reps >= 0.9:
            acid_dep(deg05,row,col)
            acid_reps -= 1
    
    base_vol = next_pct_base * 2.0
    base_reps = np.rint(base_vol * 20)
    print("base_reps " , base_reps)
    while base_reps > 0.1:
        if base_reps >= 14: 
            base_dep(deg7,row,col)
            base_reps -= 14
        elif base_reps >= 12:
            base_dep(deg6,row,col)
            base_reps -= 12
        elif base_reps >= 10:
            base_dep(deg5,row,col)
            base_reps -= 10
        elif base_reps >= 8:
            base_dep(deg4,row,col)
            base_reps -= 8
        elif base_reps >= 6:
            base_dep(deg3,row,col)
            base_reps -= 6
        elif base_reps >= 4:
            base_dep(deg2,row,col)
            base_reps -= 4
        elif base_reps >= 2:
            base_dep(deg1,row,col)
            base_reps -= 2
        elif base_reps >= 0.9:
            base_dep(deg05,row,col)
            base_reps -= 1
        
    
    # code for # of times to go to well and volume to get...
    
    #--------------------meausuring pH and cleaning probe------------------------------------------------------------------------------#
    pgoto(row,col)
    pH = pH_measure(15)
    pClean()
    
    return pH
    
    

In [ ]:
#Test Run with DI water
time.sleep(5)
target_ratio_AB = 1.0
next_idx, next_ratio, next_pct_acid, next_pct_base = find_nearest(ratio_AB,target_ratio_AB)
count = 0
make_measure_clean(next_idx, next_pct_acid, next_pct_base, count)

In [ ]:
comp_idx = np.linspace(0,30,31,dtype=int)              # list of indices
pct_acid = np.linspace(0.05,0.8,31)            # list of pct acid we can make
pct_base = np.ones(pct_acid.shape) - pct_acid  # list of pct base we can make
ratio_AB = pct_acid/pct_base                   # list of ratios we can make

def find_nearest(unmeasured_ratio_AB, target_ratio_AB):                  # Function to find the nearest ratio we can create
    next_idx = (np.abs(ratio_AB - target_ratio_AB)).argmin()
    next_ratio = ratio_AB[next_idx]
    next_pct_acid = pct_acid[next_idx]
    next_pct_base = pct_base[next_idx]
    return next_idx,next_ratio,next_pct_acid,next_pct_base


target_ratio_AB = 2.64432727
next_idx, next_ratio, next_pct_acid, next_pct_base = find_nearest(ratio_AB,target_ratio_AB)
print(next_pct_acid)
vare = np.rint(next_pct_acid*2.0*20)
print(vare)




In [ ]:
#import torch
from IPython.core.pylabtools import figsize
from scipy.stats.mstats import mquantiles
from scipy.optimize import least_squares, curve_fit

In [ ]:
#numpy version of function
def f_log_np(x, A, B):
    F = A - B * np.log10(x) 
    return F

def f_exp_np(x, A, B, C, D):
    F = A + B * np.exp(C * (x-D)) 
    return F

def f_linear_np(x, A, B, C, D):
    F = A + B + C * (x - D)
    return F

def f_quad_np(x, A, B, C):
    F = A + B * (x - C)**2  
    return F

def f_cubic_np(x, A, B, C):
    F = A + B * (x - C)**3 
    return F

def f_sin_np(x, A, B, C):
    F = A + B * np.sin(C * x)
    return F

def f_pow_np(x, A, B, C, D):
    F = A + B * (x - D)**C
    return F

#actual function
def f(x):                                     # for simulating data
    f = 4.5 + 1.0 * (x - 1.0)**2
    #f = 3.2 + 1.0 * np.sin(1.5 * x)
    #f = 4.756 - np.log10(x) 
    return f

num_func = 7

In [ ]:
# Functions for LSQ optimization

# Some constraints that help for the function fitting (i.e. non-discontinuous functions)
x0 = np.array([1.0, 1.0, 1.0, 0.0])
x1 = np.array([1.0,1.0,0.0])
x_log = np.array([1.0,1.0])
x_exp = np.array([3, 3, -0.2, 0])
x_quad = np.array([3.5, 0.03, 10])

def resid_log(x, t, y):
    return (x[0] - x[1] * np.log10(t) - y)

def resid_exp(x, t, y):
    return x[0] + x[1] * np.exp(x[2] * (t - x[3])) - y

def resid_linear(x,t,y):
    return x[0] + x[1] + x[2] * (t - x[3]) - y

def resid_quad(x, t, y):
    return x[0] + x[1] * (t - x[2])**2 - y 

def resid_cubic(x, t, y):
    return x[0] + x[1] * (t - x[2])**3 - y 

def resid_sin(x, t, y):
    return x[0] + x[1] * np.sin(x[2]*t) - y 

def resid_pow(x, t, y):
    return x[0] + x[1] * (t - x[3])**x[2] - y

In [ ]:
# Sampling Space
N = 100
X_Grid_ = np.linspace(0.0526,4,N)[:,None]    # should I use np.logspace instead??
#X_grid_ = torch.linspace(0.0526, 4, N)[:,None]

In [ ]:
def EntropyAF():
    
    count = 0
    
    simulate_val = True
    
    for i in range (2):                             #first 3 compositions are selected at random
        next_idx = np.random.choice(comp_idx) 
        next_ratio = ratio_AB[next_idx]
        next_pct_acid = pct_acid[next_idx]
        next_pct_base = pct_base[next_idx]
        print('Sample:', count + 1)
        print('Ratio of [Acid]/[Base]:', next_ratio)
        print('% Acid:', next_pct_acid)
        print('% Base:', next_pct_base)
        
        if simulate_val == False:
            pH = make_measure_clean(next_idx, next_pct_acid, next_pct_base, count) # compose first ratio in well, measure pH, clean pH probe
        else:
            noise = 0.3
            pH = f(next_ratio)+np.random.normal(0,noise)   #simulate values
            
        if i == 0:
            measured_idx = np.atleast_1d(next_idx) 
            unmeasured_idx  = np.setdiff1d(comp_idx, measured_idx).astype(int) 
            measured_ratios = np.atleast_1d(ratio_AB[next_idx])
            pH_data = np.atleast_1d(pH) 
        else:
            measured_idx = np.append(measured_idx, next_idx) 
            unmeasured_idx = np.setdiff1d(comp_idx, measured_idx).astype(int)
            measured_ratios = np.append(measured_ratios, ratio_AB[next_idx])
            pH_data = np.append(pH_data, pH)
        
        print('Indices Measured:',measured_idx )
        print('Indices Unmeasured:',unmeasured_idx)
        
        count += 1
        print(pH_data)
    
    for i in range (20):                 #using entropy acq func...
        num_measurements = count 
        x_vals = measured_ratios
        y_vals = pH_data
        
        print('Sample:', count + 1)
        print('Ratio of [Acid]/[Base]:', next_ratio)
        print('% Acid:', next_pct_acid)
        print('% Base:', next_pct_base)
        
        #LSQ optimization of candidates to fit them
        res_lsq_log = least_squares(resid_log, x_log, args=(x_vals, y_vals))   #bounds = ([-np.inf,-np.inf,0.0001,-np.inf],[np.inf,np.inf,np.inf,0])
        res_lsq_exp = least_squares(resid_exp, x_exp, bounds = (-300,300), args=(x_vals, y_vals))
        res_lsq_linear = least_squares(resid_linear, x0, bounds = (-300,300), args=(x_vals, y_vals))
        res_lsq_quad = least_squares(resid_quad, x1, bounds = (-300,300),args=(x_vals, y_vals))
        res_lsq_cubic = least_squares(resid_cubic, x1, bounds = (-300,300), args=(x_vals, y_vals))
        res_lsq_sin = least_squares(resid_sin, x1, bounds = (-300,300), args=(x_vals, y_vals))
        res_lsq_pow = least_squares(resid_pow, x0, bounds = (-300,300), args=(x_vals, y_vals)) # bounds = (-300,300),
        
        #print('\nLogarithmic Residuals:', res_lsq_log.fun)
        #print('\nExponential Residuals:', res_lsq_exp.fun)
        #print('\nLinear Residuals:', res_lsq_linear.fun)
        #print('\nQuadratic Residuals:', res_lsq_quad.fun)
        #print('\nCubic Residuals:', res_lsq_cubic.fun)
        #print('\nSinusoidal Residuals:', res_lsq_sin.fun)
        #print('\nPower Function Residuals:', res_lsq_pow.fun)
        
        #Getting the standard deviation for each function
        s_log = np.sqrt(1/(num_measurements-1)*np.sum((res_lsq_log.fun)**2))
        s_exp = np.sqrt(1/(num_measurements-1)*np.sum((res_lsq_exp.fun)**2))
        s_linear = np.sqrt(1/(num_measurements-1)*np.sum((res_lsq_linear.fun)**2))
        s_quad = np.sqrt(1/(num_measurements-1)*np.sum((res_lsq_quad.fun)**2))
        s_cubic = np.sqrt(1/(num_measurements-1)*np.sum((res_lsq_cubic.fun)**2))
        s_sin = np.sqrt(1/(num_measurements-1)*np.sum((res_lsq_sin.fun)**2))
        s_pow = np.sqrt(1/(num_measurements-1)*np.sum((res_lsq_pow.fun)**2))
        
        BIC_log = num_measurements * np.log(s_log**2) + 2 * np.log(num_measurements)
        BIC_exp = num_measurements * np.log(s_exp**2) + 4 * np.log(num_measurements)
        BIC_linear = num_measurements * np.log(s_linear**2) + 4 * np.log(num_measurements)
        BIC_quad = num_measurements * np.log(s_quad**2) + 3 * np.log(num_measurements)
        BIC_cubic = num_measurements * np.log(s_cubic**2) + 3 * np.log(num_measurements)
        BIC_sin = num_measurements * np.log(s_sin**2) + 3 * np.log(num_measurements)
        BIC_pow = num_measurements * np.log(s_pow**2) + 4 * np.log(num_measurements)
        
        font = {'family' : "Times New Roman",'weight' : 'normal','size'   : 11}
        matplotlib.rc('font', **font)
        
        fig = plt.figure(figsize=[3.,2.], dpi = 300)
        ax = fig.add_axes([0,0,1,1])
        ax.set_xlabel('Candidate', fontsize = 14)
        ax.set_ylabel('BIC, arb. units', fontsize = 14)
        funcs = ['log', 'exp', 'linear', 'quad', 'cubic', 'sin', 'power']
        sums = [BIC_log, BIC_exp, BIC_linear, BIC_quad, BIC_cubic, BIC_sin, BIC_pow]
        ax.bar(funcs,sums,color=['C0', 'C1', 'C2', 'C3', 'C4','C5','C6'])
        plt.show()
        
        
        # Graphing
        graph_entropy(x_vals,y_vals,res_lsq_log,res_lsq_exp,res_lsq_linear,res_lsq_quad,res_lsq_cubic,res_lsq_sin,res_lsq_pow,
                     s_log,s_exp,s_linear,s_quad,s_cubic,s_sin,s_pow)
        
        #Creating Distributions with And Calculating the Likelihoods
        likelihood_log = np.zeros((num_measurements))
        likelihood_exp = np.zeros((num_measurements))
        likelihood_linear = np.zeros((num_measurements))
        likelihood_quad = np.zeros((num_measurements))
        likelihood_cubic = np.zeros((num_measurements))
        likelihood_sin = np.zeros((num_measurements))
        likelihood_pow = np.zeros((num_measurements))
        
        print('num_measurements', num_measurements)
        for i in range(num_measurements):
          log_dist = lambda i: 1/(s_log*np.sqrt(2*np.pi))*np.exp(-0.5*((y_vals[i]-f_log_np(x_vals[i],res_lsq_log.x[0],res_lsq_log.x[1]))/s_log)**2)
          likelihood_log[i]= log_dist(i)
          exp_dist = lambda i: 1/(s_exp*np.sqrt(2*np.pi))*np.exp(-0.5*((y_vals[i]-f_exp_np(x_vals[i],res_lsq_exp.x[0],res_lsq_exp.x[1],res_lsq_exp.x[2], res_lsq_exp.x[3]))/s_exp)**2)
          likelihood_exp[i]= exp_dist(i)
          linear_dist = lambda i: 1/(s_linear*np.sqrt(2*np.pi))*np.exp(-0.5*((y_vals[i]-f_linear_np(x_vals[i],res_lsq_linear.x[0],res_lsq_linear.x[1],res_lsq_linear.x[2],res_lsq_linear.x[3]))/s_linear)**2)
          likelihood_linear[i]= linear_dist(i)
          quad_dist = lambda i: 1/(s_quad*np.sqrt(2*np.pi))*np.exp(-0.5*((y_vals[i]-f_quad_np(x_vals[i],res_lsq_quad.x[0],res_lsq_quad.x[1],res_lsq_quad.x[2]))/s_quad)**2)
          likelihood_quad[i]= quad_dist(i)
          cubic_dist = lambda i: 1/(s_cubic*np.sqrt(2*np.pi))*np.exp(-0.5*((y_vals[i]-f_cubic_np(x_vals[i],res_lsq_cubic.x[0],res_lsq_cubic.x[1],res_lsq_cubic.x[2]))/s_cubic)**2)
          likelihood_cubic[i]= cubic_dist(i)
          sin_dist = lambda i: 1/(s_sin*np.sqrt(2*np.pi))*np.exp(-0.5*((y_vals[i]-f_sin_np(x_vals[i],res_lsq_sin.x[0],res_lsq_sin.x[1],res_lsq_sin.x[2]))/s_sin)**2)
          likelihood_sin[i]= sin_dist(i)
          pow_dist = lambda i: 1/(s_pow*np.sqrt(2*np.pi))*np.exp(-0.5*((y_vals[i]-f_pow_np(x_vals[i],res_lsq_pow.x[0],res_lsq_pow.x[1],res_lsq_pow.x[2],res_lsq_pow.x[3]))/s_pow)**2)
          likelihood_pow[i]= pow_dist(i)
            
        #Summing the Log of Likelihoods
        log_sum_prob = np.sum(np.log(likelihood_log))
        exp_sum_prob = np.sum(np.log(likelihood_exp))
        linear_sum_prob = np.sum(np.log(likelihood_linear))
        quad_sum_prob = np.sum(np.log(likelihood_quad))
        cubic_sum_prob = np.sum(np.log(likelihood_cubic))
        sin_sum_prob = np.sum(np.log(likelihood_sin))
        pow_sum_prob = np.sum(np.log(likelihood_pow))

        #print('\nSum of Log Likelihood (Logarithmic Model)' , log_sum_prob)
        #print('Sum of Log Likelihood (Exponential Model)' , exp_sum_prob)
        #print('Sum of Log Likelihood (Linear Model)' , linear_sum_prob)
        #print('Sum of Log Likelihood (Quadratic Model)' , quad_sum_prob)
        #print('Sum of Log Likelihood (Cubic Model)' , cubic_sum_prob)
        #print('Sum of Log Likelihood (Sin Model)' , sin_sum_prob)
        #print('Sum of Log Likelihood (Power Function Model)' , pow_sum_prob)
        
        fig = plt.figure(figsize=[7.,5.], dpi = 120)
        ax = fig.add_axes([0,0,1,1])
        ax.set_xlabel('Candidate', fontsize = 14)
        ax.set_ylabel('Sum of Log-Likelihood', fontsize = 14)
        funcs = ['log', 'exp', 'linear', 'quad', 'cubic', 'sin', 'power']
        sums = [log_sum_prob, exp_sum_prob, linear_sum_prob, quad_sum_prob, cubic_sum_prob, sin_sum_prob, pow_sum_prob]
        ax.bar(funcs,sums,color=['C0', 'C1', 'C2', 'C3', 'C4','C5','C6'])
        plt.show()
        
        #Constructing the GMM based on Entropy
        weight = 1/num_func

        #pH range over which GMM will have its entropy evaluated for a given x-value
        pH_range = np.linspace(2.5,6.5,100)

        #import integration library for normalizing GMM
        import scipy.integrate as integrate

        #array to contain the entropy for each x value
        ent = np.empty_like(X_Grid_)

        plt.figure(figsize=[10.,8.], dpi = 120)

        for i in range(100):
          log_dist_GMM = lambda y: weight/(s_log*np.sqrt(2*np.pi))*np.exp(-0.5*((y-f_log_np(X_Grid_[i],res_lsq_log.x[0],res_lsq_log.x[1]))/s_log)**2)
          exp_dist_GMM = lambda y: weight/(s_exp*np.sqrt(2*np.pi))*np.exp(-0.5*((y-f_exp_np(X_Grid_[i],res_lsq_exp.x[0],res_lsq_exp.x[1],res_lsq_exp.x[2],res_lsq_exp.x[3]))/s_exp)**2)
          linear_dist_GMM = lambda y: weight/(s_linear*np.sqrt(2*np.pi))*np.exp(-0.5*((y-f_linear_np(X_Grid_[i],res_lsq_linear.x[0],res_lsq_linear.x[1],res_lsq_linear.x[2],res_lsq_linear.x[3]))/s_linear)**2)
          quad_dist_GMM = lambda y: weight/(s_quad*np.sqrt(2*np.pi))*np.exp(-0.5*((y-f_quad_np(X_Grid_[i],res_lsq_quad.x[0],res_lsq_quad.x[1],res_lsq_quad.x[2]))/s_quad)**2)
          cubic_dist_GMM = lambda y: weight/(s_cubic*np.sqrt(2*np.pi))*np.exp(-0.5*((y-f_cubic_np(X_Grid_[i],res_lsq_cubic.x[0],res_lsq_cubic.x[1],res_lsq_cubic.x[2]))/s_cubic)**2)
          sin_dist_GMM = lambda y: weight/(s_sin*np.sqrt(2*np.pi))*np.exp(-0.5*((y-f_sin_np(X_Grid_[i],res_lsq_sin.x[0],res_lsq_sin.x[1],res_lsq_sin.x[2]))/s_sin)**2)
          pow_dist_GMM = lambda y: weight/(s_pow*np.sqrt(2*np.pi))*np.exp(-0.5*((y-f_pow_np(X_Grid_[i],res_lsq_pow.x[0],res_lsq_pow.x[1],res_lsq_pow.x[2],res_lsq_pow.x[3]))/s_pow)**2)
          GMM = log_dist_GMM(pH_range)+exp_dist_GMM(pH_range)+linear_dist_GMM(pH_range)+quad_dist_GMM(pH_range)+cubic_dist_GMM(pH_range)+sin_dist_GMM(pH_range)+pow_dist_GMM(pH_range)

          plt.plot(pH_range,GMM)
          plt.xlabel('pH', fontsize = 18)
          plt.ylabel('Likelihood Values', fontsize = 18)
          plt.title('Evolution of GMM', fontsize = 18)
          entropy = 0.0
          for j in range(99):
            p = integrate.quad(lambda pH_range: log_dist_GMM(pH_range)+exp_dist_GMM(pH_range)+quad_dist_GMM(pH_range)+cubic_dist_GMM(pH_range)+sin_dist_GMM(pH_range)+pow_dist_GMM(pH_range),pH_range[j],pH_range[j+1])[0]
            entropy -= p*np.log(p)
          ent[i] = entropy

        #Maybe try formula for contiuous entropy... 

        plt.figure(figsize=[10.,8.], dpi = 120)
        plt.plot(X_Grid_,ent)
        next_point = np.argmax(ent)
        plt.vlines(X_Grid_[next_point],ent[np.argmin(ent)],ent[next_point], 'r')
        plt.title('Acquisition Function', fontsize = 18)
        plt.xlabel('[Acid]/[Base]', fontsize = 18)
        plt.ylabel('Entropy Acquisition Function', fontsize = 18)
        plt.title('Entropy At Each Potential Composition', fontsize = 18)

        plt.figure(figsize=[10.,8.], dpi = 120)
        #plt.plot(X_grid_.flatten(), f_log_np(X_grid_, A_true, B_true, C_true, D_true), label = 'True Function')
        plt.plot(X_Grid_,f_log_np(X_Grid_,res_lsq_log.x[0],res_lsq_log.x[1]), label = 'Optimized Log Model')
        plt.plot(X_Grid_,f_exp_np(X_Grid_,res_lsq_exp.x[0],res_lsq_exp.x[1],res_lsq_exp.x[2], res_lsq_exp.x[3]), label = 'Optimized Exp Model')
        plt.plot(X_Grid_,f_linear_np(X_Grid_,res_lsq_linear.x[0],res_lsq_linear.x[1],res_lsq_linear.x[2], res_lsq_linear.x[3]), label = 'Optimized Linear Model')
        plt.plot(X_Grid_,f_quad_np(X_Grid_,res_lsq_quad.x[0],res_lsq_quad.x[1], res_lsq_quad.x[2]), label = 'Optimized Quad Model')
        plt.plot(X_Grid_,f_cubic_np(X_Grid_,res_lsq_cubic.x[0],res_lsq_cubic.x[1],res_lsq_cubic.x[2]), label = 'Optimized Cubic Model')
        plt.plot(X_Grid_,f_sin_np(X_Grid_,res_lsq_sin.x[0],res_lsq_sin.x[1],res_lsq_sin.x[2]), label = 'Optimized Sinusoidal Model')
        plt.plot(X_Grid_,f_pow_np(X_Grid_,res_lsq_pow.x[0],res_lsq_pow.x[1],res_lsq_pow.x[2], res_lsq_pow.x[3]), label = 'Optimized Power Function Model')
        plt.vlines(X_Grid_[next_point],2,6, 'r')
        plt.plot(x_vals, y_vals, 'ro', label = 'data')
        plt.xlabel('[Acid]/[Base]', fontsize = 18)
        plt.ylabel('pH', fontsize = 18)
        plt.legend()

        print('The next Composition to target is: ', X_Grid_[next_point])
        
        target_ratio_AB = X_Grid_[next_point]
        next_idx, next_ratio, next_pct_acid, next_pct_base = find_nearest(ratio_AB,target_ratio_AB)
        
        if simulate_val == False:
            pH = make_measure_clean(next_idx, next_pct_acid, next_pct_base, count) # compose first ratio in well, measure pH, clean pH probe
        else:
            noise = 0.1
            pH = f(next_ratio)+np.random.normal(0,noise)   #simulate values
            
        measured_idx = np.append(measured_idx, next_idx) 
        unmeasured_idx = np.setdiff1d(comp_idx, measured_idx).astype(int)
        measured_ratios = np.append(measured_ratios, ratio_AB[next_idx])
        pH_data = np.append(pH_data, pH)
        count += 1
        
        print('Indices Measured:',measured_idx )
        print('Indices Unmeasured:',unmeasured_idx)

        



In [ ]:
def graph_entropy(x_vals,y_vals,res_lsq_log,res_lsq_exp,res_lsq_linear,res_lsq_quad,res_lsq_cubic,res_lsq_sin,res_lsq_pow,
                 s_log,s_exp,s_linear,s_quad,s_cubic,s_sin,s_pow):
    plt.figure(figsize=[10.,8.], dpi = 120)
    #plt.plot(X_Grid_.flatten(), f_log_np(X_Grid_, A_true, B_true, C_true, D_true), label = 'True Function')
    logplot = f_log_np(X_Grid_,res_lsq_log.x[0],res_lsq_log.x[1])
    plt.plot(X_Grid_,logplot, label = 'Optimized Log Model')
    plt.fill_between(X_Grid_[:,0],logplot[:,0]-1.96*s_log,logplot[:,0]+1.96*s_log, color = 'b', alpha = .2)

    #expplot = f_exp_np(X_Grid_,res_lsq_exp.x[0],res_lsq_exp.x[1],res_lsq_exp.x[2], res_lsq_exp.x[3])
    #plt.plot(X_Grid_,expplot, label = 'Optimized Exp Model')
    #plt.fill_between(X_Grid_[:,0],expplot[:,0]-1.96*s_exp,expplot[:,0]+1.96*s_exp, color = 'tab:orange', alpha = .1)

    linplot = f_linear_np(X_Grid_,res_lsq_linear.x[0],res_lsq_linear.x[1],res_lsq_linear.x[2], res_lsq_linear.x[3])
    plt.plot(X_Grid_,linplot, label = 'Optimized Linear Model')
    plt.fill_between(X_Grid_[:,0],linplot[:,0]-1.96*s_linear,linplot[:,0]+1.96*s_linear, color = 'tab:green', alpha = .03)

    quadplot = f_quad_np(X_Grid_,res_lsq_quad.x[0],res_lsq_quad.x[1], res_lsq_quad.x[2])
    plt.plot(X_Grid_,quadplot, label = 'Optimized Quad Model')
    plt.fill_between(X_Grid_[:,0],quadplot[:,0]-1.96*s_quad,quadplot[:,0]+1.96*s_quad, color = 'tab:red', alpha = .03)

    cubicplot = f_cubic_np(X_Grid_,res_lsq_cubic.x[0],res_lsq_cubic.x[1],res_lsq_cubic.x[2])
    plt.plot(X_Grid_,cubicplot, label = 'Optimized Cubic Model')
    plt.fill_between(X_Grid_[:,0],cubicplot[:,0]-1.96*s_cubic,cubicplot[:,0]+1.96*s_cubic, color = 'tab:purple', alpha = .05)

    sinplot = f_sin_np(X_Grid_,res_lsq_sin.x[0],res_lsq_sin.x[1],res_lsq_sin.x[2])
    plt.plot(X_Grid_,sinplot, label = 'Optimized Sinusoidal Model')
    plt.fill_between(X_Grid_[:,0],sinplot[:,0]-1.96*s_sin,sinplot[:,0]+1.96*s_sin, color = 'tab:brown', alpha = .05)

    powplot = f_pow_np(X_Grid_,res_lsq_pow.x[0],res_lsq_pow.x[1],res_lsq_pow.x[2], res_lsq_pow.x[3])
    plt.plot(X_Grid_,powplot, label = 'Optimized Power Function Model')
    plt.fill_between(X_Grid_[:,0],powplot[:,0]-1.96*s_pow,powplot[:,0]+1.96*s_pow, color = 'tab:brown', alpha = .2)


    plt.plot(x_vals,y_vals, 'ro', label = 'data')
    plt.xlabel('[Acid]/[Base]', fontsize = 18)
    plt.ylabel('pH', fontsize = 18)
    plt.legend()

    #print('\nLogarithmic Optimized Par_ameters A:',res_lsq_log.x[0], 'B:', res_lsq_log.x[1])
    #print('\nExponential Optimized Parameters A:',res_lsq_exp.x[0], 'B:', res_lsq_exp.x[1], 'C:', res_lsq_exp.x[2])
    #print('\nLinear Optimized Parameters A:',res_lsq_linear.x[0], 'B:', res_lsq_linear.x[1], 'C:', res_lsq_linear.x[2], 'D:', res_lsq_linear.x[3])
    #print('\nQuadratic Optimized Parameters A:',res_lsq_quad.x[0], 'B:', res_lsq_quad.x[1], 'C:', res_lsq_quad.x[2])
    #print('\nCubic Optimized Parameters A:',res_lsq_cubic.x[0], 'B:', res_lsq_cubic.x[1], 'C:', res_lsq_cubic.x[2])
    #print('\nSinusoidal Optimized Parameters A:',res_lsq_sin.x[0], 'B:', res_lsq_sin.x[1], 'C:', res_lsq_sin.x[2])
    #print('\nPower Function Optimized Parameters A:',res_lsq_pow.x[0], 'B:', res_lsq_pow.x[1], 'C:', res_lsq_pow.x[2], 'D:', res_lsq_pow.x[3])

In [ ]:
EntropyAF()